# parser

> Module to open and parse HAR files

In [ ]:
#| default_exp parser

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
import pandas as pd
from datetime import datetime, date
import numpy as np
import re

In [ ]:
#| export

url_profile = 'https://www.linkedin.com/sales/people/'
account2company_keys = {    
    'name' : 'name',
    'yearFounded' : 'founded',
    'flagshipCompanyUrl' : 'linkedin',
    'website' : 'website',
    'description' : 'description',
    'cype' : 'c_type',
    'location' :'location',
    'industry' : 'industry',
    'decisionMakersDisplayCount' : 'decisionMakersDisplayCount',
    'teamLinkConnectionsDisplayCount' : 'teamLinkConnectionsDisplayCount'
    }

In [ ]:
#| export
def parse_account_page(account_p: dict):
    company = {}
    company['linkedin_id'] = account_p['entityUrn'].split(':')[-1]
    for k,v in account_p.items():
        if k in account2company_keys.keys():
            company[account2company_keys[k]] = v
    company['keywords'] = account_p['specialties'] if 'specialties' in account_p.keys() else []
    company['updated'] = datetime.now()

    if 'revenueRange' in account_p.keys():
        company['revenue'] = parse_revenue(account_p['revenueRange'])
    else:
        company['revenue'] = 0
    if 'employeeGrowthPercentages' in account_p.keys():
        employeegrowth = account_p.pop('employeeGrowthPercentages')
        company.update({x['timespan']:x['percentage'] for x in employeegrowth})
    if 'employeeCountInfo' in account_p.keys():
        company['employees'] = account_p['employeeCountInfo']['employeeCount']
    if 'headquarters' in account_p.keys():
        hq={}
        hq['country'] = account_p['headquarters']['country'] if 'country' in account_p['headquarters'].keys() else ''
        hq['state'] = account_p['headquarters']['geographicArea'] if 'geographicArea' in account_p['headquarters'].keys() else ''
        hq['city'] = account_p['headquarters']['city'] if 'city' in account_p['headquarters'].keys() else ''
        hq['address'] = account_p['headquarters']['line1'] if 'line1' in account_p['headquarters'].keys() else ''
        hq['zipcode'] = account_p['headquarters']['postalCode'] if 'postalCode' in account_p['headquarters'].keys() else ''
        company.update(hq)
    return company

In [ ]:
#| export
def account_parse_search(account):
    """
    Hay que arreglar esta funcion para poder recuperar más información si es posible como revenue y otras, además de manejar si no encuentra los key.
    podemos generar un diccionario de correspondencia para transformar todo en los key del modelo
    """
    company = {}
    company['linkedin_id'] = account['entityUrn'].split(':')[-1]
    company['description'] = clean_text_string_func(account['description']) if 'description' in account.keys() else ''
    company['name'] = account['companyName']
    company['industry'] = account['industry'] if 'industry' in account.keys() else ''  
    return company

In [ ]:
#| export
def parse_interest_buyer(buyerinterest):
    buyerinterest.update({'linkedin_id':buyerinterest['buyer'].split(':')[-1]})
    _ = buyerinterest.pop('seller') if 'seller' in buyerinterest.keys() else ''
    _ = buyerinterest.pop('sellerCompany') if 'sellerCompany' in buyerinterest.keys() else ''
    _ = buyerinterest.pop('requestId') if 'requestId' in buyerinterest.keys() else ''
    _ = buyerinterest.pop('viewerAutogenListId') if 'viewerAutogenListId' in buyerinterest.keys() else ''
    features = buyerinterest.pop('featureCategories') if 'featureCategories' in buyerinterest.keys() else []
    while features:
        feature = features.pop()
        buyerinterest.update({feature['name']:feature['level']})
    return buyerinterest

In [ ]:
#| export
def parse_employees(employees):
    employees['linkedin_id'] = employees.pop('entityUrn').split(':')[-1]
    employees['employees'] = int(employees.pop('employeeCount'))
    _ = employees.pop('sharedAlumniSchoolIds') if 'sharedAlumniSchoolIds' in employees.keys() else ''
    _ = employees.pop('employeeDisplayCount') if 'employeeDisplayCount' in employees.keys() else ''
    _ = employees.pop('sharedAlumniDisplayCount') if 'sharedAlumniDisplayCount' in employees.keys() else '' 
    _ = employees.pop('employeeCountRange') if 'employeeCountRange' in employees.keys() else ''   
    return employees

In [ ]:
#| export
def clean_lead_keys(lead):
    _ = lead.pop('$recipeType') if '$recipeType' in lead.keys() else ''
    _ = lead.pop('$anti_abuse_metadata') if '$anti_abuse_metadata' in lead.keys() else ''
    _ = lead.pop('$anti_abuse_annotations') if '$anti_abuse_annotations' in lead.keys() else ''
    _ = lead.pop('$trackingId') if '$trackingId' in lead.keys() else ''
    _ = lead.pop('awards') if 'awards' in lead.keys() else ''
    _ = lead.pop('connectedTime') if 'connectedTime' in lead.keys() else ''
    _ = lead.pop('courses') if 'courses' in lead.keys() else ''
    _ = lead.pop('certifications') if 'certifications' in lead.keys() else ''
    _ = lead.pop('companyPictureDisplayImage') if 'companyPictureDisplayImage' in lead.keys() else ''
    _ = lead.pop('companyUrnResolutionResult') if 'companyUrnResolutionResult' in lead.keys() else ''
    _ = lead.pop('createdAt') if 'createdAt' in lead.keys() else ''
    _ = lead.pop('crmStatus') if 'crmStatus' in lead.keys() else ''
    _ = lead.pop('current') if 'current' in lead.keys() else ''
    _ = lead.pop('defaultPosition') if 'defaultPosition' in lead.keys() else ''
    _ = lead.pop('educations') if 'educations' in lead.keys() else ''
    _ = lead.pop('facePiles') if 'facePiles' in lead.keys() else ''
    _ = lead.pop('fullNamePronunciationAudio') if 'fullNamePronunciationAudio' in lead.keys() else ''
    _ = lead.pop('lastUpdatedTimeInListAt') if 'lastUpdatedTimeInListAt' in lead.keys() else ''
    _ = lead.pop('leadAssociatedAccount') if 'leadAssociatedAccount' in lead.keys() else ''
    _ = lead.pop('leadAssociatedAccountResolutionResult') if 'leadAssociatedAccountResolutionResult' in lead.keys() else ''
    _ = lead.pop('languages') if 'languages' in lead.keys() else ''
    _ = lead.pop('memorialized') if 'memorialized' in lead.keys() else ''
    _ = lead.pop('memberBadges') if 'memberBadges' in lead.keys() else ''
    _ = lead.pop('noteCount') if 'noteCount' in lead.keys() else ''
    _ = lead.pop('new') if 'new' in lead.keys() else ''
    _ = lead.pop('newListEntitySinceLastViewed') if 'newListEntitySinceLastViewed' in lead.keys() else ''
    # _ = lead.pop('objectUrn') if 'objectUrn' in lead.keys() else ''
    _ = lead.pop('openLink') if 'openLink' in lead.keys() else ''
    _ = lead.pop('organizations') if 'organizations' in lead.keys() else ''
    _ = lead.pop('outreachActivity') if 'outreachActivity' in lead.keys() else ''
    _ = lead.pop('profilePictureDisplayImage') if 'profilePictureDisplayImage' in lead.keys() else ''
    _ = lead.pop('pendingInvitation') if 'pendingInvitation' in lead.keys() else ''
    _ = lead.pop('profileUnlockInfo') if 'profileUnlockInfo' in lead.keys() else ''
    _ = lead.pop('pictureInfo') if 'pictureInfo' in lead.keys() else ''
    _ = lead.pop('posId') if 'posId' in lead.keys() else ''
    _ = lead.pop('profileBackgroundPicture') if 'profileBackgroundPicture' in lead.keys() else ''   
    _ = lead.pop('relatedColleagueCompanyId') if 'relatedColleagueCompanyId' in lead.keys() else ''
    _ = lead.pop('richMedia') if 'richMedia' in lead.keys() else ''
    _ = lead.pop('savedAccountResolutionResult') if 'savedAccountResolutionResult' in lead.keys() else ''
    _ = lead.pop('savedAccount') if 'savedAccount' in lead.keys() else ''
    _ = lead.pop('sharedConnectionsHighlight') if 'sharedConnectionsHighlight' in lead.keys() else ''
    _ = lead.pop('showTotalConnectionsPage') if 'showTotalConnectionsPage' in lead.keys() else ''   
    _ = lead.pop('trackingId') if 'trackingId' in lead.keys() else ''
    _ = lead.pop('unlocked') if 'unlocked' in lead.keys() else ''
    _ = lead.pop('volunteeringExperiences') if 'volunteeringExperiences' in lead.keys() else ''
    return lead

In [ ]:
#| export
def parse_lead_dict(lead):
    if 'profileUrnResolutionError' in lead.keys():
        print(f"Profile Error ")
        return {}
    lead['id'] = lead['objectUrn'].split(':')[-1] if 'objectUrn' in lead.keys() else ''
    if 'currentPositions' in lead.keys() and len(lead['currentPositions']) > 0:
        c_position = lead.pop('currentPositions')
        c_position = c_position[0]
        current = parse_current_position(c_position)
        lead.update(current)
        # lead['companyName'] = c_position['companyName'] if 'companyName' in c_position.keys() else None
        # lead['title'] = c_position['title'] if 'title' in c_position.keys() else None
        lead['company_location'] = c_position['location'] if 'location' in c_position.keys() else None
        # lead['company_id'] = c_position['companyUrn'].split(':')[-1] if 'companyUrn' in c_position.keys() else None
        lead.update(c_position['tenureAtCompany'])
        lead['company_industry'] = c_position['industry'] if 'industry' in c_position.keys() else None

    if lead.get('summary'):
        lead['summary'] = lead['summary']
    if lead.get('entityUrn'):
        linkedin = lead['entityUrn'] 
        lead['linkedin'] = create_link(linkedin)
    if 'teamlink' in lead.keys() and lead['teamlink']:
        tmp = lead.pop('teamlinkIntrosHighlight')
        lead['teamlink'] = tmp['count']
    else:
        lead['teamlink'] = 0
    lead['shared_conection'] = lead['sharedConnectionsHighlight']['count'] if 'sharedConnectionsHighlight' in lead.keys() else 0
    # _ = lead.pop('sharedConnectionsHighlight') if 'sharedConnectionsHighlight' in lead.keys() else ''
    lead['pastPositions'] = parse_past_positions(lead['pastPositions']) if 'pastPositions' in lead.keys() else ''
    highlights = lead.pop('spotlightHighlights') if 'spotlightHighlights' in lead.keys() else None
    while highlights:
        highlight = highlights.pop()
        if highlight['type'] == 'COMPANY_FOLLOW':
            lead['followNTT'] = 1
            lead['follow_since'] = datetime.fromtimestamp(highlight['highlight']['com.linkedin.sales.deco.desktop.searchv2.DecoratedCompanyFollowHighlight']['followedAt']/1000)
        elif highlight['type'] == 'RECENTLY_POSTED_ON_LINKEDIN':
            lead['posted'] = 1
        elif highlight['type'] == 'RECENT_POSITION_CHANGE':
            change = highlight['highlight']['com.linkedin.sales.deco.common.profile.highlights.DecoratedRecentPositionChangeHighlight']
            lead['changed_position'] = change['duration'] 
    return clean_lead_keys(lead) #lead

In [ ]:
#| export    
def parse_rlead_dict(lead):
    # print(f"RLEAD FUNCTION {lead}")
    old_positions = []
    if 'profileUrnResolutionError' in lead.keys():
        return {}
    lead = lead['profileUrnResolutionResult']
    try:
        positions = lead.pop('positions') if 'positions' in lead.keys() else []
        while positions:
            position = positions.pop()
            if position['current']:
                lead.update(parse_current_position(position))
            else:
                old_positions.append(position)
        lead['old_positions'] = "\ ".join([parse_old_position(pos) for pos in old_positions])
    except:
        raise Exception
    if lead.get('entityUrn'):
        linkedin = lead['entityUrn'] 
        lead['linkedin'] = create_link(linkedin)
    if lead.get('summary'):
        lead['summary'] = clean_text_string_func(lead['summary'])
    # lead = clean_lead_keys(lead)

    if 'teamlink' in lead.keys() and lead['teamlink']:
        tmp = lead.pop('teamlinkIntrosHighlight')
        lead['teamlink'] = tmp['count']
    else:
        lead['teamlink'] = 0
    # _ = lead.pop('profilePictureDisplayImage') if 'profilePictureDisplayImage' in lead.keys() else ''
    lead['shared_conection'] = lead['sharedConnectionsHighlight']['count'] if 'sharedConnectionsHighlight' in lead.keys() else 0
    k_flattern = []
    for k,v in lead.items():
        if isinstance(v, dict):
             k_flattern.append(k)
    for k in k_flattern:
        tmp_dict = lead.pop(k)
        lead.update(tmp_dict)
    return clean_lead_keys(lead)

In [ ]:
#| export
def create_link(entityUrn):
    matched = re.search('\((.*)\)', entityUrn)
    return f"{url_profile}{matched.groups()[0]}" if matched else entityUrn

In [ ]:
#| export
def parse_old_position(position):
    description = clean_text_string_func(position['description']) if 'description' in position.keys() else ''
    title = position['title'] if 'title' in position.keys() else ''
    started = parse_date(position['startedOn']).year if 'startedOn' in position.keys() else 0
    ended = parse_date(position['endedOn']).year if 'endedOn' in position.keys() else 1
    company = position['companyName'] if 'companyName' in position.keys() else ''
    return f"{company}; {title}; {description}; {started}-{ended}"

In [ ]:
#| export
def parse_skills(data):
    # print("#### SKILLS PARSING: ####")
    skills = {'entityUrn':data['entityUrn']}
    skills['skills'] = [skill['name'] for skill in data['skills']]
    # for skill in data['skills']:
    #     try:
    #         skills.update({skill['name']:skill['numOfEndorsement']})
    #     except:
    #         print("problem getting skills'")
    #         pass
    return skills

In [ ]:
#| export
def parse_profile_page(profile):
    # print(f"PROFILE FUNCTION {profile}")
    skills = {'entityUrn':profile['entityUrn']}
    for skill in profile['skills']:
        try:
            skills.update({skill['name']:skill['numOfEndorsement']})
        except:
            pass
    profile['skills'] = [skill['name'] for skill in profile['skills']]
    # projects = profile['projects']
    # profile = clean_lead_keys(profile)
    profile['linkedin'] = create_link(profile['entityUrn'])
    if profile.get('summary'):
        profile['summary'] = clean_text_string_func(profile.pop('summary'))
    profile['publications'] = parse_publications(profile.pop('publications'))
    profile['patents'] = parse_publications(profile.pop('patents'))
    profile['projects'] = parse_projects(profile.pop('projects'))
    current, old = parse_positions(profile.pop('positions'))
    profile.update(current)
    profile['old_positions'] = old
    return clean_lead_keys(profile), skills

In [ ]:
#| export
def parse_info_profile(profile):
    # print(f"INFO PROFILE FUNCTION {profile}")
    profile = clean_lead_keys(profile)
    contacts = parse_contactInfo(profile.pop('contactInfo'))
    profile.update(contacts)
    current, old = parse_positions(profile.pop('positions'))
    profile.update(current)
    profile['old_positions'] = old
    return profile

In [ ]:
#| export
def parse_contactInfo(contactInfo):
    contact = {}
    if contactInfo.get('primaryEmail'):
        contact['email'] = contactInfo['primaryEmail']
    for k,v in contactInfo.items():
        if k == 'socialHandles':
            for i in v:
                contact[i['type']] = i['name']
    return contact

In [ ]:
#| export
def parse_publications(publications):
    publications_list = []
    for publication in publications:
        title = publication['name'] if 'name' in publication.keys() else ''
        abstract = publication['description'] if 'description' in publication.keys() else '' 
        publications_list.append(f"{title}. {abstract}")
    return "; ".join(publications_list)

In [ ]:
#| export
def parse_projects(projects):
    projects_list = []
    for project in projects:
        started = parse_date(project['startedOn']).year if 'startedOn' in project.keys() else 1
        ended = parse_date(project['endedOn']).year if 'endedOn' in project.keys() else 1
        title = project['title'] if 'title' in project.keys() else ''
        projects_list.append(f"{title}. {started} - {ended}")
    return "; ".join(projects_list)
  

In [ ]:
#| export
  
def parse_patents(patents):
    patents_list = []
    for patent in patents:
        title = patent['title'] if 'title' in patent.keys() else ''
        patents_list.append(title)
    return "; ".join(patents_list)
 

In [ ]:
#| export
   
def parse_positions(positions):
    old_positions = []
    current = {}
    while positions:
        position = positions.pop()
        if position['current']:
            current = parse_current_position(position)
        else:
            old_positions.append(position)
    old = parse_past_positions(old_positions)
    return current, old

In [ ]:
#| export
def parse_current_position(position:dict):
    """
    Function to parse the current position. Receive a dictionary with the info of current position '$recipetype DecoratedPosition'
    return Dictionary with Keys 'company_id', 'pos_description', 'pos_started', '
    """
    # _ = position.pop('current')
    # _ = position.pop('new') if 'new' in position.keys() else ''
    # _ = position.pop('companyPictureDisplayImage') if 'companyPictureDisplayImage' in position.keys() else ''
    # _ = position.pop('companyUrnResolutionResult') if 'companyUrnResolutionResult' in position.keys() else ''
    companyUrn = position.pop('companyUrn') if 'companyUrn' in position.keys() else ':'
    position['company_id'] = companyUrn.split(':')[-1]
    # _ = position.pop('createdAt') if 'createdAt' in position.keys() else ''
    # _ = position.pop('posId') if 'posId' in position.keys() else ''
    position['company_location'] = position['location'] if 'location' in position.keys() else None
    if 'description' in position.keys():
        position['pos_description'] = position.pop('description')
    if 'startedOn' in position.keys():
        position['pos_started'] = parse_date(position.pop('startedOn')) 
    return position
  

In [ ]:
#| export
  
def parse_date(dateOn):
    datelist = []
    for k in ['year', 'month', 'day']:
        if k in dateOn.keys():
            datelist.append(dateOn[k]) 
        else:
            datelist.append(1)
    return datetime(*datelist)

In [ ]:
#| export
def parse_past_positions(positions):
    pos_list = []
    for position in positions:
        started = parse_date(position['startedOn']).year if 'startedOn' in position.keys() else 1000
        ended = parse_date(position['endedOn']).year if 'endedOn' in position.keys() else 1000
        company = position['companyName'].strip(' ') if 'companyName' in position.keys() else ''
        title = position['title'].strip(' ') if 'title' in position.keys() else ''
        pos_list.append(f"{company}; {title}; {started}-{ended}")
    return "/ ".join(pos_list)

In [ ]:
#| export
def load_har(file):
    with open(file,'r') as fin:
        tmp = json.loads(fin.read())
    return tmp
  

In [ ]:
#| export
  
def load_hars(path, recursive=False):
    tmp_j = {'log':{'entries':[]}}
    if recursive:
        files = path.glob('**/*.har')
    else:
        files = path.glob('*.har')
    for file in files:
        print(file)
        with open(file,'r') as fin:
            tmp = json.loads(fin.read())
            tmp_j['log']['entries'].extend(tmp['log']['entries'])
    return tmp_j

In [ ]:
#| export
def parse_revenue(revenueRange):
    if 'reportedRevenue' in revenueRange.keys():
        revenue = revenueRange['reportedRevenue']
        amount = revenue['amount']
        units = revenue['unit']
    else:
        amount = (revenueRange['estimatedMaxRevenue']['amount'] + revenueRange['estimatedMinRevenue']['amount'])/2
        units = revenueRange['estimatedMaxRevenue']['unit']
    if units == 'MILLION':
        return amount*1000000
    elif units == 'THOUSAND':
        return amount*1000
    elif units == 'BILLION':
        return amount * 1000000000
    else:
        return amount

In [ ]:
#| export
def parse_HAR(file=False, several=False, recursive=False, har=False):
    if har:
        tmp_j = har
    elif several:            
        tmp_j = load_hars(file, recursive)
    else:
        tmp_j = load_har(file)
    data =[]
    leads = []
    recommended_leads = []
    employee_count = []
    accounts = []
    account = []
    profiles = []
    contacts = []
    buyer_interest = []
    skills_list = []
    for entry in tmp_j['log']['entries']:
        if 'text' in entry['response']['content'].keys():# and entry['response']['content']['mimeType'] == 'application/json' :
            try:
                data_site = json.loads(entry['response']['content']['text'])
            except:
                continue
            if isinstance(data_site, (list, int)):
                continue
            if 'skills' in data_site.keys():
                skills_list.append(parse_skills(data_site))
                data.append(data_site)
            if 'account' in data_site.keys()  :
                account.append(parse_account_page(data_site))
                data.append(data_site)
            if 'seller' in data_site.keys() or 'buyer' in data_site.keys():
                buyer_interest.append(parse_interest_buyer(data_site))
            if 'recommendedLeads' in data_site.keys():
                recommended_leads.extend([parse_rlead_dict(rlead) for rlead in data_site['recommendedLeads']]) 
            if 'objectUrn' in data_site.keys() :
                contacts.append(parse_info_profile(data_site))
                data.append(data_site)
            if 'employeeCount' in data_site.keys():
                employee_count.append(parse_employees(data_site))
            if 'elements' in data_site.keys():# and 'metadata' in data_site.keys():
                elements = data_site['elements']
                if len(elements) > 0 and 'objectUrn' in elements[0].keys():
                    leads.extend([parse_lead_dict(lead) for lead in  elements])
                elif len(elements) > 0 and 'companyName' in elements[0].keys():
                    accounts.extend([account_parse_search(account) for account in elements])
                elif len(elements) > 0 and 'recommendedLeads' in elements[0].keys():
                    for element in elements:
                        recommended_leads.extend([parse_rlead_dict(rlead) for rlead in element['recommendedLeads']])

            elif 'results' in data_site.keys() :
                if len(data_site['errors'].values()) > 0 or len(data_site['results'].keys()) != 1:
                    continue
                else:
                    keys = list(data_site['results'].keys())
                    profile_data = data_site['results'][keys[0]]
                    if 'projects' not in profile_data.keys() :
                        continue
                    profile, skills = parse_profile_page(profile_data)
                    profiles.append(profile)
                    skills_list.append(skills)
            # else:
            #     pass
    
    lead_datas =[pd.DataFrame(list_data) for list_data in [profiles, leads ,recommended_leads,  contacts] if len(list_data)>0]
    account_datas =[pd.DataFrame(list_data) for list_data in [accounts, account, buyer_interest, employee_count] if len(list_data)>0]
    leads = merge(*lead_datas, on='entityUrn') if len(lead_datas) > 0 else pd.DataFrame()
    accounts = merge(*account_datas, on='linkedin_id') if len(account_datas) > 0 else pd.DataFrame()
    return leads.sort_values(sorting_columns(leads.columns)).drop_duplicates('entityUrn').rename(columns={'location':'geoRegion'}),\
            accounts.sort_values(sorting_columns_accounts(accounts.columns)).drop_duplicates('linkedin_id'), \
            pd.DataFrame(skills_list).drop_duplicates('entityUrn').fillna(0), data
   

In [ ]:
#| export
     
def sorting_columns(columns: list):
    prefered_order = ['summary', 'pos_description', 'publications', 'projects']
    filter_order = [x for x in prefered_order if x in columns]
    return filter_order

In [ ]:
#| export
def sorting_columns_accounts(columns: list):
    prefered_order = ['description', 'revenue','score']
    filter_order = [x for x in prefered_order if x in columns]
    return filter_order

In [ ]:
#| export
def merge(*data, on='entityUrn'):#leads, rleads, profiles, contacts):
    merged = data[0]
    for i in range(1,len(data)):
        merged = pd.merge(merged, data[i], on=on, how='outer', suffixes=('', f"__{i}"))
        cols1=[]
        cols2=[]
        for col in merged.columns:
            if re.search("__\d", col):
                cols2.append(col)
            else:
                cols1.append(col)
        df1 = merged[cols1]
        df2 = merged[cols2]
        df2.columns = [col.split("__")[0] for col in df2.columns]
        df1 = df1.fillna(df2)
        df1 = df1.combine(df2, longer)
        merged = df1.copy()
    return merged.reset_index(drop=True)

In [ ]:
#| export
def longer(s1, s2):
    if isinstance(s1, (int, float)):
        return s2
#     print(s2)
        if isinstance(s2, str):
            return s2
        elif s1 > s2:
            return s1
        else:
            return s2
    elif isinstance(s1, str):
        if isinstance(s2, float):
            return s1
        elif len(s1) > len(s2):
            return s1
        else:
            return s2
    elif isinstance(s1, bool):
        if s1:
            return s1
        else:
            return s2
    else:
        return s1
# def merge_accounts()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()